In [91]:
import requests
import pandas as pd
import re

## reminder: beta version was saved before I took out all the testing cells


In [112]:
def check_date(date):
    try:
        date_pattern = r'\d{4}-\d{2}-\d{2}'
        if re.match(date_pattern, date):
            if int(date[:4]) < 2019:
                return f'after={date}T12%3A02%3A36.874Z'
            else:
                return f'before={date}T12%3A02%3A36.874Z'
    except:
        return ''

In [108]:
query_date = input('enter YYYY-MM-DD for a specific date. improper date formats will show most recent songs') ## This will turn into a button on streamlit

bs_url = 'https://api.beatsaver.com/maps/latest?' 
search_date_formatted = check_date(query_date)
url_end = '&pageSize=50'


In [109]:
# gets fresh data from beat saver
response = requests.get(f'{bs_url}{search_date_formatted}{url_end}').json()

# looking at the guts

In [96]:
response['docs'][0] #ok, saving one cell that looks at the first song

{'id': '36a0d',
 'name': 'Bishu - Bomb (feat. LeyeT)',
 'description': 'bomb',
 'uploader': {'id': 4341006,
  'name': 'spinposts',
  'avatar': 'https://cdn.beatsaver.com/avatar/4341006.jpg',
  'type': 'SIMPLE',
  'admin': False,
  'curator': False,
  'playlistUrl': 'https://api.beatsaver.com/users/id/4341006/playlist'},
 'metadata': {'bpm': 150.0,
  'duration': 175,
  'songName': 'Bomb (feat. LeyeT)',
  'songSubName': '',
  'songAuthorName': 'Bishu',
  'levelAuthorName': 'spinout'},
 'stats': {'plays': 0,
  'downloads': 0,
  'upvotes': 0,
  'downvotes': 0,
  'score': 0.5},
 'uploaded': '2023-10-22T16:41:38.056531Z',
 'automapper': False,
 'ranked': False,
 'qualified': False,
 'versions': [{'hash': 'e66cd98f4ec419aab1eef53a01ee0fd73777ed88',
   'state': 'Published',
   'createdAt': '2023-10-22T16:41:33.632727Z',
   'sageScore': 3,
   'diffs': [{'njs': 18.0,
     'offset': -0.5,
     'notes': 716,
     'bombs': 0,
     'obstacles': 374,
     'nps': 4.232,
     'length': 423.0,
     'cha

# functional stuff

In [99]:
bs_data = []

In [100]:
# makes a dictionary from the data of each song in response, then adds it to the BS data list.

for i in range(len(response['docs'])):
    song = {}
    song['bs_map_id'] = response['docs'][i]['id']
    song['title'] = response['docs'][i]['metadata']['songName']
    song['artist'] = response['docs'][i]['metadata']['songAuthorName']
    song['mapper_name'] = response['docs'][i]['uploader']['name']
    song['mapper_id'] = response['docs'][i]['uploader']['id']
    song['duration_seconds'] = response['docs'][i]['metadata']['duration']
    song['auto_mapped'] = response['docs'][i]['automapper']
    song['upload_date'] = response['docs'][i]['lastPublishedAt']
    try: 
        song['tags'] = response['docs'][i]['tags']
    except:
        song['tags'] = ['none']
    song['difficulties'] = [response['docs'][i]['versions'][0]['diffs'][x]['difficulty'] for x in range(len(response['docs'][i]['versions'][0]['diffs']))]
    bs_data.append(song)


In [102]:
#turn the list into a dataframe

df = pd.DataFrame(bs_data)
df.shape

(50, 10)

In [103]:
# Normal Export, will overwrite existing data every time:
df.to_csv('beatsaver_songs.csv') 

# append export (depreciated):
# df.to_csv('beatsaver_songs.csv', mode='a', headers=False)